In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Paraná - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [5]:
data = pd.read_csv('../Inputs Atuais/2008_01_model_input_PR.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Paraná - Produção de Cimento (t)', axis=1)
data

,Ano,Paraná - Desemprego,Paraná - IDH,Paraná - IDH Renda,Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraná - Consumo de Cimento (t)
0,2008,8.033948,0.782138,0.768548,2.148670e+08,1.122835e+07,17.780948,1.878837e+08,0.745648,0.834122,...,2.868108,11.760983,17.558665,912.624708,1226.296848,18.012164,13.653739,78.027482,2.236583,3001.521000
1,2009,8.063370,0.782910,0.767137,2.163260e+08,1.133139e+07,17.787925,1.879466e+08,0.748601,0.835310,...,2.305885,9.472585,16.652522,912.466179,1217.653383,17.954863,13.668549,78.238643,2.155189,3092.068000
2,2010,8.026843,0.783594,0.766111,2.218671e+08,1.138377e+07,17.886000,1.893914e+08,0.751387,0.835940,...,1.958246,9.332485,17.928528,912.482023,1194.060192,17.703414,13.001948,76.311573,2.092804,3749.448000
3,2011,8.013806,0.784190,0.765458,2.270561e+08,1.138563e+07,17.967281,1.906364e+08,0.753998,0.836020,...,2.079088,10.985689,18.661309,912.629463,1182.416452,17.710041,12.630186,74.615878,2.043995,4174.942000
4,2012,6.472496,0.784389,0.764921,2.317175e+08,1.133915e+07,18.011636,1.916091e+08,0.755977,0.835445,...,2.183282,8.159014,19.310061,912.778766,1176.491125,17.874291,12.612094,73.865245,1.941425,4522.969000
5,2013,7.011799,0.785421,0.765581,2.358269e+08,1.122742e+07,18.040420,1.923917e+08,0.759142,0.834565,...,2.403445,7.913561,19.233593,912.914618,1170.878793,18.105264,12.720105,73.487865,1.844730,4578.716000
6,2014,7.462443,0.785706,0.765963,2.386532e+08,1.115283e+07,18.028052,1.924772e+08,0.760987,0.833112,...,2.748397,10.350442,20.725057,912.973266,1150.562966,18.275455,13.157699,74.605015,1.851460,4341.821857
7,2015,7.813261,0.785515,0.766043,2.410111e+08,1.107686e+07,18.031270,1.926648e+08,0.762152,0.831122,...,2.501529,12.454398,22.054778,913.057535,1145.712487,18.177724,13.346348,75.170150,1.905966,4108.786286
8,2016,7.840669,0.788366,0.758774,2.424341e+08,1.101317e+07,18.026717,1.926914e+08,0.771114,0.838386,...,2.073464,13.111908,22.315526,913.132268,1158.573328,18.115048,13.261122,74.043778,1.933375,3869.334857
9,2017,7.553100,0.787582,0.759026,2.428100e+08,1.097443e+07,18.022550,1.926406e+08,0.770943,0.835784,...,2.177386,9.479812,21.977949,913.165670,1160.957791,18.281050,13.470416,73.546507,1.889856,3764.518000


In [6]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Paraná - Desemprego,Paraná - IDH,Paraná - IDH Renda,Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,8.033948,0.782138,0.768548,2.148670e+08,1.122835e+07,17.780948,1.878837e+08,0.745648,0.834122,5.483335e+08,...,-7723.574973,2.868108,11.760983,17.558665,912.624708,1226.296848,18.012164,13.653739,78.027482,2.236583
1,8.063370,0.782910,0.767137,2.163260e+08,1.133139e+07,17.787925,1.879466e+08,0.748601,0.835310,5.302764e+08,...,-8961.781709,2.305885,9.472585,16.652522,912.466179,1217.653383,17.954863,13.668549,78.238643,2.155189
2,8.026843,0.783594,0.766111,2.218671e+08,1.138377e+07,17.886000,1.893914e+08,0.751387,0.835940,5.046425e+08,...,-9895.682385,1.958246,9.332485,17.928528,912.482023,1194.060192,17.703414,13.001948,76.311573,2.092804
3,8.013806,0.784190,0.765458,2.270561e+08,1.138563e+07,17.967281,1.906364e+08,0.753998,0.836020,4.854622e+08,...,-9989.170714,2.079088,10.985689,18.661309,912.629463,1182.416452,17.710041,12.630186,74.615878,2.043995
4,6.472496,0.784389,0.764921,2.317175e+08,1.133915e+07,18.011636,1.916091e+08,0.755977,0.835445,4.867661e+08,...,-9429.326328,2.183282,8.159014,19.310061,912.778766,1176.491125,17.874291,12.612094,73.865245,1.941425
5,7.011799,0.785421,0.765581,2.358269e+08,1.122742e+07,18.040420,1.923917e+08,0.759142,0.834565,5.225848e+08,...,-7157.441850,2.403445,7.913561,19.233593,912.914618,1170.878793,18.105264,12.720105,73.487865,1.844730
6,7.462443,0.785706,0.765963,2.386532e+08,1.115283e+07,18.028052,1.924772e+08,0.760987,0.833112,5.676413e+08,...,-3453.461327,2.748397,10.350442,20.725057,912.973266,1150.562966,18.275455,13.157699,74.605015,1.851460
7,7.813261,0.785515,0.766043,2.410111e+08,1.107686e+07,18.031270,1.926648e+08,0.762152,0.831122,5.179392e+08,...,-1442.919387,2.501529,12.454398,22.054778,913.057535,1145.712487,18.177724,13.346348,75.170150,1.905966
8,7.840669,0.788366,0.758774,2.424341e+08,1.101317e+07,18.026717,1.926914e+08,0.771114,0.838386,4.524144e+08,...,-3128.377228,2.073464,13.111908,22.315526,913.132268,1158.573328,18.115048,13.261122,74.043778,1.933375
9,7.553100,0.787582,0.759026,2.428100e+08,1.097443e+07,18.022550,1.926406e+08,0.770943,0.835784,4.213689e+08,...,-5386.337108,2.177386,9.479812,21.977949,913.165670,1160.957791,18.281050,13.470416,73.546507,1.889856


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     3092.068000
1     3749.448000
2     4174.942000
3     4522.969000
4     4578.716000
5     4341.821857
6     4108.786286
7     3869.334857
8     3764.518000
9     3773.354000
10    3786.974000
11    4235.532000
12    4784.413000
13    4538.256000
14            NaN
Name: Paraná - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input 

,Paraná - Desemprego,Paraná - IDH,Paraná - IDH Renda,Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,0.954430,-1.699678,1.331277,-1.806944,0.348365,-2.042958,-1.906928,-1.621982,-0.042462,1.204752,...,-0.569731,2.026352,0.984445,-1.266771,-0.975425,1.940409,-0.268956,1.137644,1.835993,2.079287
1,1.017927,-1.261317,0.939993,-1.661370,1.009092,-1.965533,-1.871184,-1.293295,0.429908,0.820657,...,-0.838980,0.085558,-0.051710,-1.745665,-1.582298,1.579454,-0.539264,1.176616,1.956737,1.461930
2,0.939098,-0.873574,0.655199,-1.108495,1.345008,-0.877208,-1.049479,-0.983091,0.680031,0.275392,...,-1.042057,-1.114490,-0.115145,-1.071300,-1.521642,0.594192,-1.725446,-0.577589,0.854816,0.988761
3,0.910963,-0.535386,0.474312,-0.590757,1.356922,0.024771,-0.341452,-0.692424,0.712066,-0.132596,...,-1.062386,-0.697344,0.633403,-0.684027,-0.957222,0.107944,-1.694185,-1.555909,-0.114802,0.618558
4,-2.415320,-0.422527,0.325283,-0.125660,1.058841,0.516968,0.211766,-0.472052,0.483303,-0.104860,...,-0.940648,-0.337667,-0.646476,-0.341164,-0.385670,-0.139500,-0.919352,-1.603519,-0.544022,-0.159410
5,-1.251457,0.163252,0.508405,0.284361,0.342383,0.836388,0.656850,-0.119710,0.133711,0.657046,...,-0.446625,0.422337,-0.757614,-0.381577,0.134391,-0.373874,0.170234,-1.319280,-0.759813,-0.892823
6,-0.278927,0.325010,0.614316,0.566357,-0.135907,0.699136,0.705450,0.085716,-0.443735,1.615453,...,0.358807,1.613108,0.345772,0.406656,0.358905,-1.222272,0.973093,-0.167721,-0.121014,-0.841773
7,0.478169,0.216545,0.636582,0.801625,-0.623089,0.734849,0.812173,0.215483,-1.234902,0.558230,...,0.796000,0.760923,1.298413,1.109410,0.681498,-1.424830,0.512055,0.328724,0.202137,-0.428361
8,0.537319,1.834498,-1.380170,0.943598,-1.031518,0.684323,0.827306,1.213246,1.652199,-0.835563,...,0.429496,-0.716758,1.596124,1.247214,0.967586,-0.887756,0.216390,0.104443,-0.441935,-0.220470
9,-0.083280,1.389158,-1.310222,0.981108,-1.279958,0.638087,0.798387,1.194161,0.618336,-1.495939,...,-0.061498,-0.358019,-0.048438,1.068806,1.095455,-0.788179,0.999483,0.655217,-0.726280,-0.550555


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     3092.068000
1     3749.448000
2     4174.942000
3     4522.969000
4     4578.716000
5     4341.821857
6     4108.786286
7     3869.334857
8     3764.518000
9     3773.354000
10    3786.974000
11    4235.532000
Name: Paraná - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Paraná - Desemprego,Paraná - IDH,Paraná - IDH Renda,Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-0.734266,-1.069894,-1.42466,0.465024,-0.914446,-0.223777,0.114326,1.035451,-2.637193,1.108405,...,0.964765,-1.482779,-2.260447,0.964616,0.339314,-0.540552,0.224215,0.839814,-2.237536,-1.155068


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    4784.413
Name: Paraná - Consumo de Cimento (t), dtype: float64

In [12]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [13]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [16]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2540110221, 2494480914, 3083572124, 1419067499, 2181521461, 1901259622, 206045376, 3837421211, 3859939312, 3243813208, 3009694649, 3934274260, 1497747982, 1531917824, 1076911713, 2304990847, 3538118870, 190864949, 1280686890, 1717946746, 3393768655, 1672401575, 3897693772, 2488230145, 1007475838, 2553204436, 2566903650, 1565595737, 1010365409, 1200328153, 3235850340, 2370073496, 321441648, 102672761, 476013237, 1719470595, 1560259688, 693984609, 3898925505, 2452160006, 1563836550, 1306485867, 4272623051, 2970842647, 326359142, 1313299919, 3462335700, 1163098523, 470187107, 397378991, 1423609929, 807733008, 3588563066, 4164624931, 2139389507, 2320140713, 4012033471, 2702536496, 3098782045, 155597413, 2299926687, 2515009624, 1867727936, 1971421734, 2301107379, 1834675338, 905406194, 1053939263, 998190735, 2304999875, 3337097860, 2609951789, 4103230981, 3903845737, 3832044594, 2802083987, 1186185638, 1342480866, 2274270093, 1846577306, 646374102, 1672932967, 967938635, 2808952830, 766676

val_loss: 47239.578125


Step: 88 ___________________________________________
val_loss: 73512.078125


Step: 89 ___________________________________________
val_loss: 41581.34375


Step: 90 ___________________________________________
val_loss: 74987.4609375


Step: 91 ___________________________________________
val_loss: 67570.0390625


Step: 92 ___________________________________________
val_loss: 84397.203125


Step: 93 ___________________________________________
val_loss: 92680.8515625


Step: 94 ___________________________________________
val_loss: 92197.921875


Step: 95 ___________________________________________
val_loss: 77024.140625


Step: 96 ___________________________________________
val_loss: 51662.203125


Step: 97 ___________________________________________
val_loss: 39365.81640625


Step: 98 ___________________________________________
val_loss: 88836.7421875


Step: 99 ___________________________________________
val_loss: 87858.984375


final_seed: 3235850340


In [17]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 472ms/step - loss: 16155888.0000 - val_loss: 17624040.0000
Epoch 2/10000
1/1 [==============================] - 0s 16ms/step - loss: 15866475.0000 - val_loss: 19917684.0000
Epoch 3/10000
1/1 [==============================] - 0s 15ms/step - loss: 15461312.0000 - val_loss: 18934222.0000
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 15421293.0000 - val_loss: 17412136.0000
Epoch 5/10000
1/1 [==============================] - 0s 13ms/step - loss: 14843343.0000 - val_loss: 18013650.0000
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 14430413.0000 - val_loss: 16882392.0000
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 14306564.0000 - val_loss: 16697080.0000
Epoch 8/10000
1/1 [==============================] - 0s 13ms/step - loss: 13754237.0000 - val_loss: 15605424.0000
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 13635180.0000

1/1 [==============================] - 0s 14ms/step - loss: 3695461.7500 - val_loss: 5879880.0000
Epoch 74/10000
1/1 [==============================] - 0s 14ms/step - loss: 3613061.2500 - val_loss: 5901412.0000
Epoch 75/10000
1/1 [==============================] - 0s 14ms/step - loss: 3538828.7500 - val_loss: 5784199.0000
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 3460980.7500 - val_loss: 5586689.5000
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 3384319.2500 - val_loss: 5645473.5000
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 3368902.2500 - val_loss: 5494644.0000
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 3242408.7500 - val_loss: 5384090.5000
Epoch 80/10000
1/1 [==============================] - 0s 14ms/step - loss: 3243719.2500 - val_loss: 5392373.5000
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 3155963.2500 - val_loss: 5056806.500

1/1 [==============================] - 0s 13ms/step - loss: 1200191.8750 - val_loss: 2271202.7500
Epoch 146/10000
1/1 [==============================] - 0s 13ms/step - loss: 1076297.6250 - val_loss: 2106308.0000
Epoch 147/10000
1/1 [==============================] - 0s 13ms/step - loss: 1067060.1250 - val_loss: 2167993.7500
Epoch 148/10000
1/1 [==============================] - 0s 13ms/step - loss: 1035904.8125 - val_loss: 2014155.5000
Epoch 149/10000
1/1 [==============================] - 0s 14ms/step - loss: 1037538.0000 - val_loss: 2095178.5000
Epoch 150/10000
1/1 [==============================] - 0s 13ms/step - loss: 1021201.3125 - val_loss: 2050582.7500
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 998914.0625 - val_loss: 1979458.0000
Epoch 152/10000
1/1 [==============================] - 0s 13ms/step - loss: 994638.4375 - val_loss: 2009385.1250
Epoch 153/10000
1/1 [==============================] - 0s 13ms/step - loss: 964011.8125 - val_loss: 194642

1/1 [==============================] - 0s 13ms/step - loss: 420917.5000 - val_loss: 903387.1250
Epoch 218/10000
1/1 [==============================] - 0s 13ms/step - loss: 408984.3438 - val_loss: 895865.0000
Epoch 219/10000
1/1 [==============================] - 0s 13ms/step - loss: 408282.6562 - val_loss: 906325.3750
Epoch 220/10000
1/1 [==============================] - 0s 14ms/step - loss: 408473.7188 - val_loss: 880063.8750
Epoch 221/10000
1/1 [==============================] - 0s 13ms/step - loss: 391096.3750 - val_loss: 865617.6250
Epoch 222/10000
1/1 [==============================] - 0s 13ms/step - loss: 400775.0000 - val_loss: 886471.0000
Epoch 223/10000
1/1 [==============================] - 0s 14ms/step - loss: 401313.2812 - val_loss: 850982.3750
Epoch 224/10000
1/1 [==============================] - 0s 14ms/step - loss: 400190.2500 - val_loss: 866111.5000
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 384297.5000 - val_loss: 856756.3125
Epoch 22

1/1 [==============================] - 0s 13ms/step - loss: 205262.3906 - val_loss: 468412.9062
Epoch 291/10000
1/1 [==============================] - 0s 13ms/step - loss: 231679.5000 - val_loss: 481030.9688
Epoch 292/10000
1/1 [==============================] - 0s 13ms/step - loss: 228155.2031 - val_loss: 470990.6562
Epoch 293/10000
1/1 [==============================] - 0s 13ms/step - loss: 216083.3906 - val_loss: 441645.9062
Epoch 294/10000
1/1 [==============================] - 0s 13ms/step - loss: 227380.5625 - val_loss: 468530.2188
Epoch 295/10000
1/1 [==============================] - 0s 14ms/step - loss: 201424.2500 - val_loss: 454072.6250
Epoch 296/10000
1/1 [==============================] - 0s 13ms/step - loss: 212848.3750 - val_loss: 480301.7812
Epoch 297/10000
1/1 [==============================] - 0s 13ms/step - loss: 223200.1719 - val_loss: 443516.9688
Epoch 298/10000
1/1 [==============================] - 0s 13ms/step - loss: 215596.0469 - val_loss: 423390.4062
Epoch 29

...

1/1 [==============================] - 0s 14ms/step - loss: 147460.4219 - val_loss: 113885.3203
Epoch 1741/10000
1/1 [==============================] - 0s 14ms/step - loss: 150820.3125 - val_loss: 117361.9453
Epoch 1742/10000
1/1 [==============================] - 0s 14ms/step - loss: 147400.7031 - val_loss: 116988.5469
Epoch 1743/10000
1/1 [==============================] - 0s 14ms/step - loss: 147450.6250 - val_loss: 114050.3281
Epoch 1744/10000
1/1 [==============================] - 0s 13ms/step - loss: 147449.6094 - val_loss: 114516.4766
Epoch 1745/10000
1/1 [==============================] - 0s 14ms/step - loss: 147431.4219 - val_loss: 114821.7031
Epoch 1746/10000
1/1 [==============================] - 0s 14ms/step - loss: 147395.7344 - val_loss: 115214.5000
Epoch 1747/10000
1/1 [==============================] - 0s 14ms/step - loss: 147438.1719 - val_loss: 114557.7891
Epoch 1748/10000
1/1 [==============================] - 0s 14ms/step - loss: 147360.1719 - val_loss: 117958.2031


1/1 [==============================] - 0s 13ms/step - loss: 147463.4844 - val_loss: 114161.0000
Epoch 1813/10000
1/1 [==============================] - 0s 13ms/step - loss: 150975.7031 - val_loss: 103216.4609
Epoch 1814/10000
1/1 [==============================] - 0s 13ms/step - loss: 147740.7500 - val_loss: 120442.7188
Epoch 1815/10000
1/1 [==============================] - 0s 13ms/step - loss: 147420.0000 - val_loss: 114526.5547
Epoch 1816/10000
1/1 [==============================] - 0s 13ms/step - loss: 151370.2812 - val_loss: 102162.7969
Epoch 1817/10000
1/1 [==============================] - 0s 13ms/step - loss: 150306.7031 - val_loss: 107696.6172
Epoch 1818/10000
1/1 [==============================] - 0s 13ms/step - loss: 147536.4688 - val_loss: 119609.9297
Epoch 1819/10000
1/1 [==============================] - 0s 13ms/step - loss: 145355.6094 - val_loss: 119313.4062
Epoch 1820/10000
1/1 [==============================] - 0s 13ms/step - loss: 151321.7969 - val_loss: 101196.5234


1/1 [==============================] - 0s 14ms/step - loss: 151210.9531 - val_loss: 102863.3359
Epoch 1885/10000
1/1 [==============================] - 0s 14ms/step - loss: 148193.1719 - val_loss: 125131.4609
Epoch 1886/10000
1/1 [==============================] - 0s 14ms/step - loss: 147426.9531 - val_loss: 116283.0000
Epoch 1887/10000
1/1 [==============================] - 0s 14ms/step - loss: 147359.0000 - val_loss: 116401.5781
Epoch 1888/10000
1/1 [==============================] - 0s 14ms/step - loss: 147341.0781 - val_loss: 118479.3203
Epoch 1889/10000
1/1 [==============================] - 0s 14ms/step - loss: 147335.6719 - val_loss: 116629.5859
Epoch 1890/10000
1/1 [==============================] - 0s 14ms/step - loss: 147462.7344 - val_loss: 121187.2734
Epoch 1891/10000
1/1 [==============================] - 0s 14ms/step - loss: 147503.4844 - val_loss: 120292.7969
Epoch 1892/10000
1/1 [==============================] - 0s 13ms/step - loss: 147327.2969 - val_loss: 117728.8984


1/1 [==============================] - 0s 13ms/step - loss: 147319.4219 - val_loss: 117645.6484
Epoch 1957/10000
1/1 [==============================] - 0s 13ms/step - loss: 147350.5781 - val_loss: 116282.6641
Epoch 1958/10000
1/1 [==============================] - 0s 13ms/step - loss: 148347.8125 - val_loss: 123346.0391
Epoch 1959/10000
1/1 [==============================] - 0s 13ms/step - loss: 147381.2500 - val_loss: 116292.4922
Epoch 1960/10000
1/1 [==============================] - 0s 13ms/step - loss: 147362.6250 - val_loss: 119585.1016
Epoch 1961/10000
1/1 [==============================] - 0s 13ms/step - loss: 151319.7969 - val_loss: 101988.8516
Epoch 1962/10000
1/1 [==============================] - 0s 13ms/step - loss: 147848.5156 - val_loss: 121467.3984
Epoch 1963/10000
1/1 [==============================] - 0s 13ms/step - loss: 147341.7500 - val_loss: 117171.6641
Epoch 1964/10000
1/1 [==============================] - 0s 13ms/step - loss: 151254.7969 - val_loss: 102882.2891


1/1 [==============================] - 0s 13ms/step - loss: 147329.2031 - val_loss: 117788.3828
Epoch 2029/10000
1/1 [==============================] - 0s 13ms/step - loss: 147316.5625 - val_loss: 117913.4297
Epoch 2030/10000
1/1 [==============================] - 0s 13ms/step - loss: 151155.0781 - val_loss: 102729.3281
Epoch 2031/10000
1/1 [==============================] - 0s 13ms/step - loss: 148079.6406 - val_loss: 124418.6094
Epoch 2032/10000
1/1 [==============================] - 0s 13ms/step - loss: 147338.5469 - val_loss: 118842.1250
Epoch 2033/10000
1/1 [==============================] - 0s 13ms/step - loss: 150909.2656 - val_loss: 101345.0781
Epoch 2034/10000
1/1 [==============================] - 0s 13ms/step - loss: 148009.1094 - val_loss: 123372.2734
Epoch 2035/10000
1/1 [==============================] - 0s 13ms/step - loss: 147383.7031 - val_loss: 116180.4531
Epoch 2036/10000
1/1 [==============================] - 0s 13ms/step - loss: 147352.9531 - val_loss: 116787.3906


In [18]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 48ms/step
[4784.413](test_target) - [[4147.2935]](prediction) = 637.1195429687496


In [19]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [20]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Paraná - Desemprego,Paraná - IDH,Paraná - IDH Renda,Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.470460,-1.248978,1.283752,-0.935098,-1.334379,-0.778595,-0.751752,-1.236383,-1.329522,1.148205,...,1.277766,1.308949,1.412346,0.333480,1.408380,1.000190,0.909918,0.683149,0.581225,1.275014
1,1.390219,0.049998,-0.128080,-0.451251,0.261510,-0.633122,-0.661502,0.023618,0.247295,0.140893,...,-0.114016,-0.190804,-0.643250,-1.356947,-0.815318,0.365766,0.482610,0.730797,0.825914,-0.107644
2,-0.919759,1.198980,-1.155672,1.386349,1.072868,1.411717,1.413253,1.212765,1.082228,-1.289098,...,-1.163750,-1.118144,-0.769096,1.023467,-0.593062,-1.365956,-1.392529,-1.413946,-1.407140,-1.167370


train_target:


,Paraná - Consumo de Cimento (t)
0,3092.068
1,3749.448
2,4174.942


test_input:


...

,Paraná - Desemprego,Paraná - IDH,Paraná - IDH Renda,Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH Educacao,Paraná - IDH Longevidade,Contratação Comercial (Bi R$),...,NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-0.734266,-1.069894,-1.42466,0.465024,-0.914446,-0.223777,0.114326,1.035451,-2.637193,1.108405,...,0.964765,-1.482779,-2.260447,0.964616,0.339314,-0.540552,0.224215,0.839814,-2.237536,-1.155068


test_target:


,Paraná - Consumo de Cimento (t)
12,4784.413


1/1 [==============================] - 0s 39ms/step
Error: 637.1195429687496




[4174.923828125,
 4349.69873046875,
 4506.064453125,
 4296.18896484375,
 3788.692138671875,
 3869.538330078125,
 3826.51904296875,
 3829.341064453125,
 3768.415283203125,
 4147.29345703125]

In [21]:
display(targets)
display(predictions)

[4522.969,
 4578.716,
 4341.821857142857,
 4108.786285714286,
 3869.334857142857,
 3764.518,
 3773.354,
 3786.974,
 4235.532,
 4784.413]

[4174.923828125,
 4349.69873046875,
 4506.064453125,
 4296.18896484375,
 3788.692138671875,
 3869.538330078125,
 3826.51904296875,
 3829.341064453125,
 3768.415283203125,
 4147.29345703125]

In [22]:
mae = mean_absolute_error(predictions, targets)
mae

231.41391322544638

In [23]:
porcentage = mae/np.mean(targets)
porcentage

0.05540669244960798